# Databricks Lakebase Inventory App - Automated Deployment

This notebook automates the deployment of the Lakebase Inventory Management System.

## What This Notebook Does:
1. Creates a Lakebase (Managed Postgres) database instance
2. Creates a Databricks App
3. Creates a Secret Scope with Flask secret key
4. Adds the Lakebase database as an app resource
5. Clones the GitHub repository to the workspace
6. Creates an app.yaml configuration file
7. Deploys the Databricks App
8. Provides redeployment commands

## Prerequisites:
- Update the variables in 0Setup notebook.
- Unity Catalog enabled workspace
- Appropriate permissions to create databases, apps, and secret scopes
- Databricks SDK installed


In [0]:
%run "../0 - SETUP/0 - Setup"

In [0]:
import secrets
import time
import json
import re
from datetime import timedelta, datetime
from time import sleep
import requests
import os

## Step 1: Create Lakebase (Managed Postgres) Database Instance

This creates a managed PostgreSQL database instance in Unity Catalog.


In [0]:
print("Step 1: Creating Lakebase Database Instance...\n")

try:
    # Create the Lakebase database instance
    # Using the correct SDK method: create_database_instance_and_wait
    from databricks.sdk.service.database import DatabaseInstance
    
    database_instance = DatabaseInstance(
        name=LAKEBASE_DB_NAME,
        capacity=LAKEBASE_CAPACITY  # Valid values: "CU_1", "CU_2", "CU_4", "CU_8"
    )
    
    print(f"Creating database '{LAKEBASE_DB_NAME}' with capacity: {LAKEBASE_CAPACITY}")
    
    # Create and wait for the database instance to be ready
    lakebase_db = w.database.create_database_instance_and_wait(
        database_instance=database_instance,
        timeout=timedelta(minutes=20)
    )
    
    print(f"✓ Lakebase database '{LAKEBASE_DB_NAME}' created successfully!")
    print(f"  Database Name: {lakebase_db.name}")
    print(f"  Status: {lakebase_db.state}")
    
    # Store database details
    LAKEBASE_DB_ID = lakebase_db.name
    
except Exception as e:
    # Check if database already exists
    if "already exists" in str(e).lower() or "AlreadyExists" in str(e):
        print(f" Database '{LAKEBASE_DB_NAME}' already exists. Using existing database.")
        # List databases and find ours
        databases = list(w.database.list_database_instances())
        lakebase_db = next((db for db in databases if db.name == LAKEBASE_DB_NAME), None)
        if lakebase_db:
            LAKEBASE_DB_ID = lakebase_db.name
            print(f"  Database Name: {LAKEBASE_DB_ID}")
            print(f"  Status: {lakebase_db.state}")
        else:
            raise Exception(f"Could not find database '{LAKEBASE_DB_NAME}'")
    else:
        print(f"✗ Error creating Lakebase database: {e}")
        raise


## Step 2: Generate Flask Secret Key


In [0]:
print("Step 2: Generating Flask Secret Key...\n")

# Generate a secure random secret key for Flask
FLASK_SECRET_KEY = secrets.token_hex(32)

print(f"✓ Flask secret key generated successfully!")
print(f"  Key length: {len(FLASK_SECRET_KEY)} characters")


## Step 3: Create Secret Scope and Add Flask Secret Key


In [0]:
print("Step 3: Creating Secret Scope...\n")

try:
    # Create secret scope
    w.secrets.create_scope(scope=SECRET_SCOPE_NAME)
    print(f"Secret scope '{SECRET_SCOPE_NAME}' created successfully!")
except Exception as e:
    if "already exists" in str(e).lower():
        print(f"Secret scope '{SECRET_SCOPE_NAME}' already exists. Using existing scope.")
    else:
        print(f"Error creating secret scope: {e}")
        raise

# Add Flask secret key to the scope
try:
    w.secrets.put_secret(
        scope=SECRET_SCOPE_NAME,
        key=SECRET_KEY_NAME,
        string_value=FLASK_SECRET_KEY
    )
    print(f"Secret key '{SECRET_KEY_NAME}' added to scope successfully!")
except Exception as e:
    print(f"Error adding secret: {e}")
    raise

# Verify the secret was created
secrets_list = list(w.secrets.list_secrets(scope=SECRET_SCOPE_NAME))
print(f"\nSecrets in scope '{SECRET_SCOPE_NAME}':")
for secret in secrets_list:
    print(f"  - {secret.key}")


## Step 4: Clone GitHub Repository to Workspace


In [0]:
print("Step 4: Cloning GitHub Repository...\n")

try:
    # Create the repo
    repo = w.repos.create(
        url=REPO_URL,
        provider=REPO_PROVIDER
    )
    
    print(f"  Repository cloned successfully!")
    print(f"  Repository ID: {repo.id}")
    print(f"  Repository Path: {repo.path}")
    print(f"  Branch: {repo.branch}")
    print(f"  URL: {repo.url}")
    
    REPO_WORKSPACE_PATH = repo.path
    
except Exception as e:
    if "already exists" in str(e).lower():
        print(f" Repository already exists at '{REPO_PATH}'. Using existing repository.")
        # Get the existing repo
        try:
            repo = w.repos.get(repo_id=REPO_PATH)
            REPO_WORKSPACE_PATH = repo.path
            print(f"  Repository Path: {REPO_WORKSPACE_PATH}")
        except:
            REPO_WORKSPACE_PATH = REPO_PATH
            print(f"  Using path: {REPO_WORKSPACE_PATH}")
    else:
        print(f" Error cloning repository: {e}")
        raise


## Step 5: Create app.yaml Configuration File


In [0]:
print("Step 5: Creating app.yaml Configuration File...\n")

# Create app.yaml content
app_yaml_content = f"""command: ["python", "app.py"]
env:
  - name: 'DATABRICKS_HOST'
    value: '{workspace_url}'
  - name: 'PGDATABASE'
    value: '{PGDATABASE}'
  - name: 'POSTGRES_SCHEMA'
    value: '{POSTGRES_SCHEMA}'
  - name: 'PGPORT'
    value: "5432"
  - name: 'PGSSLMODE'
    value: "require"
  - name: 'PGAPPNAME'
    value: "inventory_app"
  # Schema and table configuration
  - name: 'POSTGRES_TABLE'
    value: "{POSTGRES_TABLE}"
  - name: 'POSTGRES_CATEGORY_TABLE'
    value: "{POSTGRES_CATEGORY_TABLE}"
  - name: 'POSTGRES_WAREHOUSE_TABLE'
    value: "{POSTGRES_WAREHOUSE_TABLE}"
  - name: 'POSTGRES_SUPPLIER_TABLE'
    value: "{POSTGRES_SUPPLIER_TABLE}"
  - name: 'POSTGRES_SKU_TABLE'
    value: "{POSTGRES_SKU_TABLE}"
  - name: 'FORCE_DATA_RESET'
    value: "{FORCE_DATA_RESET}"
  - name: 'LOAD_SAMPLE_DATA'
    value: "{LOAD_SAMPLE_DATA}"
  - name: 'MODEL_ENDPOINT_NAME'
    value: "demand-forecasting-endpoint"
"""

# Write app.yaml to the repository path
app_yaml_path = f"{REPO_WORKSPACE_PATH}/app.yaml"

try:
    # Import workspace API
    from databricks.sdk.service.workspace import ImportFormat
    
    # Write the file using workspace API
    w.workspace.upload(
        path=app_yaml_path,
        content=app_yaml_content.encode('utf-8'),
        format=ImportFormat.AUTO,
        overwrite=True
    )
    
    print(f"  app.yaml created successfully!")
    print(f"  Path: {app_yaml_path}")
    print(f"\nConfiguration:")
    print(f"  Database: {PGDATABASE}")
    print(f"  Schema: {POSTGRES_SCHEMA}")
    print(f"  Table: {POSTGRES_TABLE}")
    print(f"  Load Sample Data: {LOAD_SAMPLE_DATA}")
    
except Exception as e:
    print(f" Error creating app.yaml: {e}")
    print(f"\napp.yaml content (copy manually if needed):\n")
    print(app_yaml_content)
    raise


## Step 6: Create Databricks App


In [0]:
print("Step 6: Creating Databricks App with Resources...\n")

try:
    # Check if app already exists first
    try:
        app = w.apps.get(name=APP_NAME)
        print(f"  App '{APP_NAME}' already exists. Using existing app.")
        print(f"  App Name: {app.name}")
        if hasattr(app, 'status'):
            print(f"  App Status: {app.app_status}")
    except Exception as get_error:
        # App doesn't exist, create it using REST API
        print(f"App does not exist. Creating new app '{APP_NAME}' with resources...")
        
        import requests
        
        # Prepare the REST API request
        api_url = f"{workspace_url}/api/2.0/apps"
        
        # Get authentication token from workspace client
        # The workspace client uses the notebook's authentication context
        headers = w.config.authenticate()
        
        # Prepare the payload with app configuration and resources
        payload = {
            "name": APP_NAME,
            "description": APP_DESCRIPTION,
            "resources": [
                {
                    "name": "database",
                    "description": "Lakebase PostgreSQL database for inventory management",
                    "database": {
                        "database_name": PGDATABASE,
                        "instance_name": LAKEBASE_DB_NAME,
                        "permission": "CAN_CONNECT_AND_CREATE"
                    }
                },
                {
                    "name": "secrets",
                    "description": "Flask secret key for session management",
                    "secret": {
                        "scope": SECRET_SCOPE_NAME,
                        "key": SECRET_KEY_NAME,
                        "permission": "READ"
                    }
                }
            ]
        }
        
        # Make the REST API call to create the app
        response = requests.post(
            api_url,
            headers=headers,
            json=payload
        )
        
        if response.status_code in [200, 201]:
            app_response = response.json()
            print(f"  Databricks App '{APP_NAME}' created successfully with resources!")
            print(f"  App Name: {app_response.get('name')}")
            print(f"  Resources attached:")
            print(f"    - Database: {LAKEBASE_DB_NAME} (Permission: CAN_CONNECT)")
            print(f"    - Secret Scope: {SECRET_SCOPE_NAME}/{SECRET_KEY_NAME} (Permission: READ)")
        else:
            print(f"API call returned status {response.status_code}")
            print(f"Response: {response.text}")
            raise Exception("App creation failed")
    
except Exception as e:
    print(f" Error with app creation: {e}")
    raise e



## Step 7: Create Managed Database Catalog

This creates a Unity Catalog catalog that connects to your Lakebase database instance.


In [0]:
print("Step 7: Creating Managed Database Catalog...\n")

try:
    # Create the database catalog
    catalog = DatabaseCatalog(
        name=CATALOG_NAME,
        database_instance_name=LAKEBASE_DB_NAME,
        database_name=PGDATABASE,
        create_database_if_not_exists=CREATE_CATALOG_IF_NOT_EXISTS
    )
    
    print(f"Creating catalog '{CATALOG_NAME}' for database instance '{LAKEBASE_DB_NAME}'...")
    
    # Create the catalog
    created_catalog = w.database.create_database_catalog(catalog=catalog)
    
    print(f"  Database catalog '{CATALOG_NAME}' created successfully!")
    print(f"  Catalog Name: {created_catalog.name}")
    print(f"  Database Instance: {created_catalog.database_instance_name}")
    print(f"  Logical Database: {created_catalog.database_name}")
    
except Exception as e:
    # Check if catalog already exists
    if "already exists" in str(e).lower() or "AlreadyExists" in str(e):
        print(f" Catalog '{CATALOG_NAME}' already exists. Using existing catalog.")
        try:
            existing_catalog = w.database.get_database_catalog(name=CATALOG_NAME)
            print(f"  Catalog Name: {existing_catalog.name}")
            print(f"  Database Instance: {existing_catalog.database_instance_name}")
            print(f"  Logical Database: {existing_catalog.database_name}")
        except Exception as get_error:
            print(f"  Note: Could not retrieve catalog details: {get_error}")
    else:
        print(f" Error creating database catalog: {e}")
        raise


## Step 8: Load and Parameterize Dashboard Template

This loads the dashboard template and replaces the catalog name with your configured value.


In [0]:
print("Step 8: Loading and Parameterizing Dashboard Template...\n")

try:
    # Read the dashboard template from the workspace
    template_path = f"/Workspace{REPO_WORKSPACE_PATH}/{DASHBOARD_TEMPLATE_PATH}"
    
    print(f"Reading dashboard template from: {template_path}")
    
    # Download the file from workspace
    file_info = w.workspace.download(path=template_path)
    dashboard_template = json.loads(file_info.read())
    
    print(f"✓ Dashboard template loaded successfully")
    
    # Convert template to JSON string for regex replacement
    dashboard_json_str = json.dumps(dashboard_template, indent=2)
    
    # Find the current catalog name in the template (pattern: catalog_name.schema_name.table_name)
    # We'll look for patterns like "reynoldspravindev_inventory_live.inventory_app.table_name"
    catalog_pattern = r'([a-zA-Z0-9_]+)\.inventory_app\.'
    
    # Find what catalog is currently in the template
    match = re.search(catalog_pattern, dashboard_json_str)
    if match:
        old_catalog = match.group(1)
        print(f"  Found existing catalog in template: {old_catalog}")
        
        # Replace all occurrences of the old catalog with the new one
        dashboard_json_str = re.sub(
            rf'{old_catalog}\.{POSTGRES_SCHEMA}\.', 
            f'{CATALOG_NAME}.{POSTGRES_SCHEMA}.', 
            dashboard_json_str
        )
        
        print(f"  Replaced '{old_catalog}' with '{CATALOG_NAME}'")
    else:
        print(f"  ⚠ Could not find catalog pattern in template. Using template as-is.")
    
    # Convert back to JSON object
    dashboard_definition = json.loads(dashboard_json_str)
    
    print(f"✓ Dashboard template parameterized successfully")
    print(f"  Catalog: {CATALOG_NAME}")
    print(f"  Schema: {POSTGRES_SCHEMA}")
    
except Exception as e:
    print(f"✗ Error loading/parameterizing dashboard template: {e}")
    raise

## Step 9: Create or Update Databricks Lakeview Dashboard
This creates a new dashboard or updates an existing one if a dashboard ID is provided in app.yaml.

**Note:** This step might run longer if the workspace has a large number of AI/BI Dashboards

In [0]:
print("Step 9: Creating/Updating Databricks Lakeview Dashboard...\n")

try:
    # Check if dashboard ID exists in app.yaml
    app_yaml_path = f"/Workspace{REPO_WORKSPACE_PATH}/app.yaml"
    existing_dashboard_id = None
    
    try:
        app_yaml_content = w.workspace.download(path=app_yaml_path).contents.read().decode('utf-8')
        
        # Extract DASHBOARD_ID from app.yaml
        dashboard_id_match = re.search(r"DASHBOARD_ID['\"]?\s*\n\s*value:\s*['\"]([^'\"]+)['\"]?", app_yaml_content)
        existing_dashboard_id = dashboard_id_match.group(1) if dashboard_id_match and dashboard_id_match.group(1) else None
        
        if existing_dashboard_id and existing_dashboard_id.strip():
            print(f"Found existing dashboard ID in app.yaml: {existing_dashboard_id}")
        else:
            existing_dashboard_id = None
            print("No dashboard ID found in app.yaml.")
    except:
        print("Could not read app.yaml.")
    
    # If no dashboard ID in app.yaml, check if a dashboard with this name already exists in workspace
    if not existing_dashboard_id:
        print(f"Checking workspace for existing dashboard with name '{DASHBOARD_DISPLAY_NAME}'...")
        try:
            # List all dashboards and find one with matching display name
            all_dashboards = list(w.lakeview.list())
            for dashboard in all_dashboards:
                if dashboard.display_name == DASHBOARD_DISPLAY_NAME:
                    existing_dashboard_id = dashboard.dashboard_id
                    print(f" Found existing dashboard in workspace: {existing_dashboard_id}")
                    break
            
            if not existing_dashboard_id:
                print("No existing dashboard found in workspace. Will create new dashboard.")
        except Exception as list_error:
            print(f"Could not list dashboards: {list_error}")
            print("Will attempt to create new dashboard.")
    
    # Determine warehouse ID
    if WAREHOUSE_ID is None:
        # Get the first available SQL warehouse
        warehouses = list(w.warehouses.list())
        if warehouses:
            warehouse_id = warehouses[0].id
            print(f"Using warehouse: {warehouses[0].name} (ID: {warehouse_id})")
        else:
            raise Exception("No SQL warehouses found. Please create one or specify WAREHOUSE_ID.")
    else:
        warehouse_id = WAREHOUSE_ID
        print(f"Using specified warehouse ID: {warehouse_id}")
    
    # Create or update dashboard using Lakeview API
    if existing_dashboard_id:
        # Update existing dashboard
        print(f"\nUpdating existing dashboard '{existing_dashboard_id}'...")
        
        try:
            # Get the current dashboard
            current_dashboard = w.lakeview.get(dashboard_id=existing_dashboard_id)
            
            # Update the dashboard with correct API signature
            updated_dashboard = w.lakeview.update(
                dashboard_id=existing_dashboard_id,
                dashboard=Dashboard(
                    display_name=DASHBOARD_DISPLAY_NAME,
                    serialized_dashboard=json.dumps(dashboard_definition),
                    warehouse_id=warehouse_id
                )
            )
            
            # Publish the dashboard
            published = w.lakeview.publish(dashboard_id=existing_dashboard_id)
            
            dashboard_id = existing_dashboard_id
            print(f"Dashboard updated and published successfully!")
            
        except Exception as update_error:
            print(f"Could not update existing dashboard: {update_error}")
            print(f"Will continue to use existing dashboard without updating...")
            dashboard_id = existing_dashboard_id
    
    if not existing_dashboard_id:
        # Create new dashboard
        print(f"\nCreating new dashboard '{DASHBOARD_DISPLAY_NAME}'...")
        
        # Create the dashboard
        new_dashboard = w.lakeview.create(
            Dashboard(
                display_name=DASHBOARD_DISPLAY_NAME,
                serialized_dashboard=json.dumps(dashboard_definition),
                warehouse_id=warehouse_id,
                parent_path=f"/Users/{current_user.user_name}"
            )
        )
        
        dashboard_id = new_dashboard.dashboard_id
        
        # Publish the dashboard
        published = w.lakeview.publish(dashboard_id=dashboard_id)
        
        print(f"Dashboard created and published successfully!")
    
    print(f"  Dashboard ID: {dashboard_id}")
    print(f"  Dashboard URL: {workspace_url}/sql/dashboardsv3/{dashboard_id}")
    
    # Store dashboard ID for next step
    CREATED_DASHBOARD_ID = dashboard_id
    
except Exception as e:
    print(f"Error creating/updating dashboard: {e}")
    print(f"\nNote: Make sure you have permissions to create dashboards and that the warehouse is running.")
    raise


## Step 10: Update app.yaml with Dashboard ID

This updates the app.yaml file with the dashboard ID so the app can embed the dashboard.


In [0]:
print("Step 10: Updating app.yaml with Dashboard ID...\n")

try:
    # Read current app.yaml
    app_yaml_path = f"/Workspace{REPO_WORKSPACE_PATH}/app.yaml"
    app_yaml_content = w.workspace.download(path=app_yaml_path).read().decode('utf-8')
    
    print(f"Reading app.yaml from: {app_yaml_path}")
    
    # Update DASHBOARD_ID value
    # Pattern matches: - name: 'DASHBOARD_ID'\n    value: 'old_value'
    dashboard_pattern = r"(- name: ['\"]DASHBOARD_ID['\"]?\s*\n\s*value: ['\"]?)([^'\"]*)([\'\"]?)"
    
    if re.search(dashboard_pattern, app_yaml_content):
        # Replace existing dashboard ID
        updated_yaml = re.sub(
            dashboard_pattern,
            rf"\g<1>{CREATED_DASHBOARD_ID}\g<3>",
            app_yaml_content
        )
        print(f"  Updated existing DASHBOARD_ID value")
    else:
        # Add DASHBOARD_ID if it doesn't exist
        updated_yaml = app_yaml_content.rstrip() + f"\n  - name: 'DASHBOARD_ID'\n    value: '{CREATED_DASHBOARD_ID}'\n"
        print(f"  Added DASHBOARD_ID to app.yaml")
    
    # Write updated app.yaml back to workspace
    w.workspace.upload(
        path=app_yaml_path,
        content=updated_yaml.encode('utf-8'),
        format=ImportFormat.AUTO,
        overwrite=True
    )
    
    print(f"  app.yaml updated successfully!")
    print(f"  Dashboard ID: {CREATED_DASHBOARD_ID}")
    
except Exception as e:
    print(f" Error updating app.yaml: {e}")
    print(f"\nPlease manually update app.yaml with:")
    print(f"  - name: 'DASHBOARD_ID'")
    print(f"    value: '{CREATED_DASHBOARD_ID}'")
    raise


## Step 11: Deploy Databricks App

This deploys the Databricks App with the updated app.yaml configuration for dashboard embedding.

**Note**: The app domain needs to be [configured](https://docs.databricks.com/aws/en/ai-bi/admin/embed#-control-allowed-embed-destinations) for the embedding to work.


In [0]:
sleep(360) # Give time for App to be created

print("Step 11: Deploying Databricks App...\n")
try:
    print(f"Deploying app '{APP_NAME}' with updated configuration...")
    
    # Create deployment configuration
    app_deployment = AppDeployment(
        source_code_path="/Workspace"+REPO_WORKSPACE_PATH
    )
    
    # Deploy the app
    deployment = w.apps.deploy_and_wait(
        app_name=APP_NAME,
        app_deployment=app_deployment,
        timeout=timedelta(minutes=20)
    )
    
    print(f"  App deployment completed!")
    print(f"  Deployment ID: {deployment.deployment_id}")
    print(f"  Status: {deployment.status}")
    
    # Get app status and URL
    app_status = w.apps.get(name=APP_NAME)
    if hasattr(app_status, 'url') and app_status.url:
        print(f"  App URL: {app_status.url}")
        print(f"\n  Your app is now deployed with the embedded dashboard!")
        print(f"\nAccess your app at: {app_status.url}")
        print(f"View the dashboard at: {workspace_url}/sql/dashboardsv3/{CREATED_DASHBOARD_ID}")
    
except Exception as e:
    print(f" Error Deploying app: {e}")
    print(f"\nYou can manually redeploy using the command in the cell below.")
    raise e


### NEXT STEPS
#### Follow below steps to enable Genie on the AI/BI Dashboard that got created in order to use Genie from the Databricks Inventory App.
- Navigate to the created dashboard named "Lakebase Inventory Dashboard"
- Click Edit Draft.
- In the right panel, click Settings. Or, click Kebab menu icon. the kebab menu > Settings to open settings.
- Click General.
- The Enable Genie toggle is off by default. Turn the toggle on to allow dashboard viewers to use the associated Genie space.
- (Optional) To link an existing Genie space, select the Link existing Genie space radio button and paste in the associated URL. If you don't select this option, Databricks automatically generates a new Genie space based on your dashboard when you publish.
- Click publish and select "unpublish"
- Now re-publish the dashboard.

## 🎉 Deployment Complete!

Your BI dashboard has been successfully deployed! Here's what was created:

1. **Database Catalog**: Unity Catalog catalog connected to your Lakebase instance
2. **Lakeview Dashboard**: Interactive dashboard with your inventory analytics
3. **App Configuration**: Updated app.yaml with dashboard ID
4. **App Redeployment**: Your app is now running with the embedded dashboard

### Next Steps:

- Access your app and navigate to the Dashboard page to see the embedded analytics
- Customize the dashboard by editing it in the Databricks SQL UI
- Share the dashboard with your team using Databricks permissions
- Proceed to **1.1GenerateSyntheticSalesData** notebook

### Troubleshooting:

If the dashboard doesn't appear in your app:
1. Verify the dashboard ID in app.yaml matches the created dashboard
2. Ensure your app has permissions to access the SQL warehouse
3. Check that the catalog and schema names match your Lakebase configuration
4. Manually redeploy using the command below if needed
